# Technical Portfolio Analyzer

This Jupyter Notebook will outline the exploration, developement and clean-up of the appliation code

### Table of Contents
- Data connectors
- Data query and clean-up
- Analytical methods
    - Technical
    - Fundamental
    - Traditional
- Visualization
- Dashboard (GUI)
    - GUI
    - Data filtration methods

### Dependencies

In [1]:
# Data analytics
import pandas as pd
import numpy as np
import panel as pn
import bs4 as bs

# #Visualization
# pn.extension('plotly')
# import plotly.express as px
# import hvplot.pandas
# import matplotlib.pyplot as plt

# System
import os
import time
from pathlib import Path
from dotenv import load_dotenv
import requests

# Finance
import alpaca_trade_api as tradeapi

import warnings
warnings.filterwarnings('ignore')

## Data Connections
- Static Data Connections
- Dynamic Data Connections

#### Static Data Connections

In [2]:
# Get tickers within S&P500 index - test dataset
sp500_tickers_path = Path('resources/sp500_tickers.csv')
sp500_tickers = pd.read_csv(sp500_tickers_path)
sp500_tickers.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",8/9/1976,66740,1902
1,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",3/31/1964,1800,1888
2,ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/2012,1551152,2013 (1888)
3,ABMD,Abiomed,Health Care,Health Care Equipment,"Danvers, Massachusetts",5/31/2018,815094,1981
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",7/6/2011,1467373,1989


#### Dynamic Data Connections

In [3]:
# Alpaca API connector
load_dotenv('/Users/ludovicschneider/Bootcamp/LS.env')

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
api = tradeapi.REST(
alpaca_api_key,
alpaca_secret_key,
api_version = "v2"
)

type(alpaca_api_key)

str

In [4]:
# Crypto connector URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=USD"

# Build out the crypto connector here

## Data Parsing

### Stock Data

In [5]:
# Get prices for tickers withing a given index or sector
def stock_prices(tickers_df, start_date, end_date):
    '''Returns pd.DataFrame with prices for the given tickers
    
    ...
    
    Parameters
    ----------
    tickers : pd.DataFrame - contains tickers for given index or sector under 
        the "Symbol" column which is the DataFrame key
    start_date : str() - string with date in following format YYYY-MM-DD
    end_date: str() - string with date in following format YYYY-MM-DD 
    
    
    Returns
    -------
    result_df : pd.DataFrame with securities price data
    '''
    
    # Get list of tickers from the tickers_df DataFrame
    tickers = list(tickers_df["Symbol"])
    
    # Parse start and end dates
    start_date = pd.Timestamp(start_date, tz="America/New_York").isoformat()
    end_date = pd.Timestamp(end_date, tz="America/New_York").isoformat()
    
    # Connect to Alpaca API and get data
    """Condition handling: 
        a. Alpaca API 422 Client Error if more than 100 tickers are passed - COMPLETE
        b. Alpaca API data max row limit of 1000 - PENDING"""
    
    
    # a. Alpaca API condition handling, sending 100 tickers at a time
    # Declate a pd.DataFrame
    result_df = pd.DataFrame()
    
    for i in range(0, len(tickers), 50):
        # Slice the ticker list into lists of 50 tickers
        sliced_tickers = tickers[i:i + 50] 
        
        temp_df = api.get_barset(
        sliced_tickers,
        timeframe = "1D",
        start = start_date,
        end = end_date,
        limit = 1000).df

        # Append temporary dataframe to result_df
        result_df = pd.concat([result_df, temp_df], axis = "columns", join = "outer")
        time.sleep(0.1)
        
    return result_df
    
    
# Method test
start_date = '2020-01-01'
end_date = '2020-02-28'
tickers_df = sp500_tickers
stocks = stock_prices(tickers_df, start_date, end_date)
stocks.to_csv("resources/price_data.csv")
stocks.head()

A                                  AAL  \
                            open   high    low  close   volume   open   
time                                                                    
2020-01-02 05:00:00+00:00  85.90  86.35  85.20  85.95  1199810  28.98   
2020-01-03 05:00:00+00:00  84.67  85.33  84.50  84.53   895182  28.27   
2020-01-06 05:00:00+00:00  84.00  84.82  83.60  84.78  1380173  27.19   
2020-01-07 05:00:00+00:00  83.96  85.26  83.94  85.09  1192756  27.56   
2020-01-08 05:00:00+00:00  85.96  86.47  85.20  85.91  1453322  27.21   

                                                            ...   ZION  \
                              high    low  close    volume  ...   open   
time                                                        ...          
2020-01-02 05:00:00+00:00  29.2950  28.65  29.09   5292996  ...  52.36   
2020-01-03 05:00:00+00:00  28.2900  27.34  27.65  13006571  ...  51.41   
2020-01-06 05:00:00+00:00  27.4901  27.08  27.32   5383583  ...  51.08   
2020-01-07 05:00:00+00:00  27.6800  27.06  27.22   5570129  ...  50.79   
2020-01-08 05:00:00+00:00  28.0900  27.10  27.84   9479470  ...  50.90   

                                                              ZTS           \
                             high     low  close   volume    open     high   
time                                                                         
2020-01-02 05:00:00+00:00  52.480  51.790  52.44  1307412  132.05  134.280   
2020-01-03 05:00:00+00:00  51.890  51.160  51.70  1012303  132.48  134.910   
2020-01-06 05:00:00+00:00  51.680  50.805  51.07  1048073  133.78  134.065   
2020-01-07 05:00:00+00:00  51.125  50.610  50.74  1223552  133.00  134.810   
2020-01-08 05:00:00+00:00  51.490  50.630  51.26  2096814  133.76  135.270   

                                                     
                              low    close   volume  
time                                                 
2020-01-02 05:00:00+00:00  131.48  134.150  1308668  
2020-01-03 05:00:00+00:00  132.27  134.110  1038786  
2020-01-06 05:00:00+00:00  132.71  133.100  1259478  
2020-01-07 05:00:00+00:00  132.67  133.600  1047293  
2020-01-08 05:00:00+00:00  133.24  133.325  1379814  

[5 rows x 2525 columns]

## Computational Methods

### Technical

In [28]:
# RSI method : Calculate the RSI indicator

def rsi_table (df, days):
    '''Returns a pd.DataFrame with Relative Strength Index (RSI) column appended
        RSI formula = 100 – (100 / (1 + RS) or can also use 100 * up / (Up + Down)
        Where RS (relative strengh)  = Up / Down
        Where Up = rolling average price up over the time window obeserved
        Where Down = rolling average price down over the time window obeserved
    
        Parameters
        ----------
        df : pd.DataFrame - dataframe to be processed
        days : int() - numbers of days for RSI calcualtion
    
        Returns
        -------
        result_df : pd.DataFrame - dataframe with RSI column appended, calcualted daily for 
        timeperiod specified by days
    '''
    
    # Filter datafrance to clumn "close" only to facilitate the calculs
    stocks_close = df.iloc[:,df.columns.get_level_values(1)=='close']
    # Swap the column multilevel index to facilitate the calculs
    stocks_close = stocks_close.swaplevel(0,1,axis=1)
    
    # Calculate the movement on the price compared to the previous day closing price
    movement_1d = stocks_close - stocks_close.shift(1)
    movement_1d.rename(columns={'close':'movement_1d'}, level=0, inplace=True)

    # Define a sub-function to calculate the RSI
    def rsi (price):
        up = price[price>0].mean()
        down = abs(price[price<0]).mean()
        return 100 * up / (up + down)
        
    # Calculate the RSI and add it to a dataframe
    rsi_df = movement_1d.rolling(window=days).apply(rsi)
    rsi_df.rename(columns={'movement_1d':'RSI'}, level=0, inplace=True)
    
    # Remove the first dates that return NaN from the rolling days calculation
    rsi_df = rsi_df[days:]

    return rsi_df

# Method test
df = stocks
days = 14
rsi_df = rsi_table (df, days)
rsi_df.to_csv("resources/rsi.csv")
rsi_df.head()

RSI                                   \
                                   A        AAL        AAP       AAPL   
time                                                                    
2020-01-23 05:00:00+00:00  44.365446  33.797217  46.196822  58.344230   
2020-01-24 05:00:00+00:00  49.661759  35.714286  45.135303  63.133834   
2020-01-27 05:00:00+00:00  44.147995  29.109589  45.191313  52.479405   
2020-01-28 05:00:00+00:00  45.316785  27.090084  46.519055  53.699151   
2020-01-29 05:00:00+00:00  42.143839  30.113348  48.351405  54.789411   

                                                                       \
                                ABBV        ABC       ABMD        ABT   
time                                                                    
2020-01-23 05:00:00+00:00  40.352282  58.705882  45.775773  51.724138   
2020-01-24 05:00:00+00:00  37.837838  52.185735  46.224247  57.236842   
2020-01-27 05:00:00+00:00  35.992579  50.307692  44.763769  57.373632   
2020-01-28 05:00:00+00:00  36.759829  49.514936  47.662606  60.000000   
2020-01-29 05:00:00+00:00  37.157472  49.393414  42.646581  63.176895   

                                                 ...                        \
                                 ACN       ADBE  ...         WY       WYNN   
time                                             ...                         
2020-01-23 05:00:00+00:00  37.103482  67.319149  ...  56.255626  49.082859   
2020-01-24 05:00:00+00:00  32.987747  76.410355  ...  57.387247  45.781711   
2020-01-27 05:00:00+00:00  31.527162  65.590100  ...  52.340426  34.884244   
2020-01-28 05:00:00+00:00  44.233753  66.224719  ...  50.074460  35.769714   
2020-01-29 05:00:00+00:00  43.757953  62.390041  ...  52.086137  40.183377   

                                                                       \
                                 XEL       XLNX        XYL        YUM   
time                                                                    
2020-01-23 05:00:00+00:00  83.343040  58.759646  54.060325  53.532251   
2020-01-24 05:00:00+00:00  83.284629  60.792431  50.961538  49.193548   
2020-01-27 05:00:00+00:00  75.516693  56.670342  45.331433  46.229632   
2020-01-28 05:00:00+00:00  74.411765  55.060034  45.629820  47.032475   
2020-01-29 05:00:00+00:00  61.674528  38.214286  52.149321  51.963439   

                                                                       
                                 ZBH       ZBRA       ZION        ZTS  
time                                                                   
2020-01-23 05:00:00+00:00  58.389913  56.394453  34.973638  50.943078  
2020-01-24 05:00:00+00:00  53.451720  58.575620  33.690745  42.343225  
2020-01-27 05:00:00+00:00  50.241158  50.545240  35.214314  44.708838  
2020-01-28 05:00:00+00:00  47.829824  53.542781  35.728953  49.350122  
2020-01-29 05:00:00+00:00  47.148455  61.150512  35.054044  42.813280  

[5 rows x 505 columns]

In [7]:
# MACD method : Calculate the MACD indicator with its Signal line

def macd_table(df, short_window, long_window, signal_window):
    ''' Moving Average Convergence Divergence (MACD) 
        Returns a pd.DataFrame with MACD and its Signal line column appended
        MACD formula = (12-day EMA - 26-day EMA)
        Signal line = MACD 9-day EMA
    
        Parameters
        ----------
        df : pd.DataFrame - dataframe to be processed
        short_ema : int() - short-term EMA for MACD calculation => default should be 12 days
        long_ema : int() - long-term EMA for MACD calculation => defaultshould be 26 days

        Returns
        -------
        result_df : pd.DataFrame - dataframe with MACD and Signal line column appended, calcualted daily for 
        timeperiod specified by days
    '''

    # Filter datafrance to clumn "close" only to facilitate the calculs
    stocks_close = df.iloc[:,df.columns.get_level_values(1)=='close']
    stocks_close = stocks_close.swaplevel(0,1,axis=1)
    
    # Define a sub-function to calculate the MACD
    def macd (price):
        short_ema = price.ewm(span=short_window, adjust=False).mean()
        long_ema = price.ewm(span=long_window, adjust=False).mean()
        macd_value = short_ema - long_ema
        return macd_value
    
    # Calculate the MACD and add it to a dataframe
    macd = stocks_close.apply(macd)
    macd.rename(columns={'close':'MACD'}, level=0, inplace=True)

    # Calculate the Signal line value
    signal = macd.ewm(span=9, adjust=False).mean()
    signal.rename(columns={'MACD':'MACD_Signal'}, level=0, inplace=True)
    
    # Concatenate/append both indicators to a new dataframe
    result_df = pd.concat([macd, signal], axis=1, join='inner')

    return result_df

# Method test
df = stocks
short_window = 12
long_window = 26
signal_window = 9
combined_macd = macd_table(df, short_window, long_window, signal_window)
combined_macd.to_csv("resources/macd.csv")
combined_macd.head()

MACD                                          \
                                  A       AAL       AAP      AAPL      ABBV   
time                                                                          
2020-01-02 05:00:00+00:00  0.000000  0.000000  0.000000  0.000000  0.000000   
2020-01-03 05:00:00+00:00 -0.113276 -0.114872  0.006382 -0.255271 -0.071795   
2020-01-06 05:00:00+00:00 -0.180792 -0.229887 -0.200885 -0.260907 -0.068993   
2020-01-07 05:00:00+00:00 -0.206899 -0.325356 -0.510158 -0.382805 -0.108290   
2020-01-08 05:00:00+00:00 -0.159582 -0.346987 -0.890241 -0.084660 -0.085993   

                                                                             \
                                ABC      ABMD       ABT       ACN      ADBE   
time                                                                          
2020-01-02 05:00:00+00:00  0.000000  0.000000  0.000000  0.000000  0.000000   
2020-01-03 05:00:00+00:00 -0.087749 -0.164330 -0.087749 -0.031111 -0.220969   
2020-01-06 05:00:00+00:00 -0.054986  0.691585 -0.116410 -0.161228 -0.232030   
2020-01-07 05:00:00+00:00 -0.078149  1.457199 -0.174234 -0.621107 -0.254006   
2020-01-08 05:00:00+00:00 -0.029992  1.909607 -0.190430 -0.938435  0.083465   

                           ... MACD_Signal                                \
                           ...          WY      WYNN       XEL      XLNX   
time                       ...                                             
2020-01-02 05:00:00+00:00  ...    0.000000  0.000000  0.000000  0.000000   
2020-01-03 05:00:00+00:00  ...    0.000638 -0.033664  0.004786 -0.037972   
2020-01-06 05:00:00+00:00  ...    0.000844 -0.091213  0.010875 -0.130851   
2020-01-07 05:00:00+00:00  ...   -0.002266 -0.149978  0.015362 -0.217400   
2020-01-08 05:00:00+00:00  ...   -0.005530 -0.191724  0.017936 -0.299257   

                                                                             \
                                XYL       YUM       ZBH      ZBRA      ZION   
time                                                                          
2020-01-02 05:00:00+00:00  0.000000  0.000000  0.000000  0.000000  0.000000   
2020-01-03 05:00:00+00:00  0.006701 -0.005425 -0.007179 -0.048501 -0.011806   
2020-01-06 05:00:00+00:00  0.009577 -0.014750 -0.031230 -0.094277 -0.040418   
2020-01-07 05:00:00+00:00  0.004937 -0.023126 -0.066437 -0.160298 -0.083236   
2020-01-08 05:00:00+00:00  0.000286 -0.027703 -0.078560 -0.376221 -0.124227   

                                     
                                ZTS  
time                                 
2020-01-02 05:00:00+00:00  0.000000  
2020-01-03 05:00:00+00:00 -0.000638  
2020-01-06 05:00:00+00:00 -0.017755  
2020-01-07 05:00:00+00:00 -0.036286  
2020-01-08 05:00:00+00:00 -0.059037  

[5 rows x 1010 columns]

In [8]:
# Bollinger Bands method : 
def bbands_table(df, length, numstd):
    '''Bollinger Bands (BB)
    returns average, upper band, and lower band
    
    Parameters
    ----------
    df : pd.DataFrame - dataframe to be processed
    lenght : int() - numer of prices we want to use to observe the average price 
    numstd : int() - number of Standard deviation we want to use to calculate the bands
    
    Returns
    -------
    result_df : pd.DataFrame - dataframe with RSI column appended, calcualted daily for 
    timeperiod specified by days
    '''
    
    # Filter datafrance to clumn "close" only to facilitate the calculs
    stocks_close = df.iloc[:,df.columns.get_level_values(1)=='close']
    stocks_close = stocks_close.swaplevel(0,1,axis=1)


    def bb_upband(price):
        #avg = pd.stats.moments.rolling_mean(price,length)
        avg = price.rolling(window= length).mean()
        #std = pd.stats.moments.rolling_std(price,length)
        std = price.rolling(window= length).std()

        upband = avg + (std*numstd)
        return np.round(upband,3)

    def bb_dnband(price):
        #avg = pd.stats.moments.rolling_mean(price,length)
        avg = price.rolling(window= length).mean()
        #std = pd.stats.moments.rolling_std(price,length)
        std = price.rolling(window= length).std()

        dnband = avg - (std*numstd)
        return np.round(dnband,3)

    bb_avg_df = stocks_close.rolling(window= length).mean()
    bb_avg_df.rename(columns={'close':'BB_Avg'}, level=0, inplace=True)
    
    bb_upband_df = stocks_close.apply(bb_upband)
    bb_upband_df.rename(columns={'close':'BB_Upband'}, level=0, inplace=True)

    bb_dnband_df = stocks_close.apply(bb_upband)
    bb_dnband_df.rename(columns={'close':'BB_Downband'}, level=0, inplace=True)

    # Concatenate/append both values to the original dataframe
    result_df = pd.concat([bb_avg_df, bb_dnband_df, bb_upband_df], axis=1, join='inner')

    return result_df



# Method test
df = stocks
length = 30
numstd = 2
combined_bb = bbands_table(df, length, numstd)
combined_bb.to_csv("resources/bb.csv")
combined_bb.tail()

BB_Avg                                  \
                                   A        AAL      AAP        AAPL   
time                                                                   
2020-02-24 05:00:00+00:00  86.032000  28.011167  140.221  317.612203   
2020-02-25 05:00:00+00:00  85.710667  27.871163  139.840  316.869203   
2020-02-26 05:00:00+00:00  85.398667  27.701830  139.482  316.059870   
2020-02-27 05:00:00+00:00  85.045333  27.470163  138.986  314.738203   
2020-02-28 05:00:00+00:00  84.660000  27.185830  138.411  313.509537   

                                                                        \
                                ABBV        ABC        ABMD        ABT   
time                                                                     
2020-02-24 05:00:00+00:00  89.026500  91.346667  179.299000  88.456000   
2020-02-25 05:00:00+00:00  89.030833  91.544667  178.295000  88.315667   
2020-02-26 05:00:00+00:00  89.028500  91.797333  177.826000  88.190667   
2020-02-27 05:00:00+00:00  88.933333  91.722000  177.201667  87.972333   
2020-02-28 05:00:00+00:00  88.812000  91.486667  176.289000  87.628000   

                                                   ... BB_Upband           \
                                  ACN        ADBE  ...        WY     WYNN   
time                                               ...                      
2020-02-24 05:00:00+00:00  210.210667  359.873000  ...    31.761  152.595   
2020-02-25 05:00:00+00:00  209.861333  360.141667  ...    31.832  152.899   
2020-02-26 05:00:00+00:00  209.397333  360.325667  ...    31.937  152.773   
2020-02-27 05:00:00+00:00  208.687667  360.080333  ...    32.112  151.841   
2020-02-28 05:00:00+00:00  207.796000  360.164333  ...    32.244  150.879   

                                                                               \
                              XEL     XLNX     XYL      YUM      ZBH     ZBRA   
time                                                                            
2020-02-24 05:00:00+00:00  72.734  105.731  90.149  108.287  162.886  259.249   
2020-02-25 05:00:00+00:00  72.522  105.369  90.048  108.690  162.890  261.439   
2020-02-26 05:00:00+00:00  72.308  105.047  90.004  109.019  162.999  263.167   
2020-02-27 05:00:00+00:00  72.156  104.776  90.045  110.156  163.743  264.901   
2020-02-28 05:00:00+00:00  72.439  104.217  90.441  111.624  164.893  266.487   

                                            
                             ZION      ZTS  
time                                        
2020-02-24 05:00:00+00:00  51.465  145.813  
2020-02-25 05:00:00+00:00  51.376  145.863  
2020-02-26 05:00:00+00:00  51.213  145.878  
2020-02-27 05:00:00+00:00  51.044  146.060  
2020-02-28 05:00:00+00:00  51.014  146.274  

[5 rows x 1515 columns]

# Filters

In [36]:
# RSI filter
# Retrieve all the tickers that match the desired requirements : 
    
def rsi_filter(df, lower_level, upper_level, start_date, end_date):
    
    '''RSI Filter
        returns list of tickers that matches conditons set by the user.
    
        Parameters
        ----------
        df : pd.DataFrame - dataframe to be processed
        lower_level: int() - price level we want to use to define the oversold zone
        upper_level: int() - price level we want to use to define the overbought zone
        start_date : str() - string with date in following format YYYY-MM-DD
        end_date : str() - string with date in following format YYYY-MM-DD
    
        Returns
        -------
        tickers : list[] - list of tickers with a RSI that was above the upper_level or below the lower_level within
        the timeperiod specified by start_date and end_date
    ''' 
    
    # Filter the rsi dataframe generated by the "rsi_table"
    rsi_df_date = df.loc[start_date : end_date, :]
    # Transpose the filtered dataframe to facilite the slicing/calculation
    rsi_df_date = rsi_df_date.transpose()
    # Remove the multiindex first level ('RSI')
    rsi_df_date = rsi_df_date.droplevel(0)
    
    # Iterate through the dataframe to create two new dataframes 
    # in which we will store the tickers being above or below the defined levels
    condition_up = rsi_df_date >= upper_level
    condition_dn = rsi_df_date <= lower_level
    
    # Retrieve the tickers which had a RSI above the upper_level over the timeperiod
    rsi_tickers_up = rsi_df_date[condition_up].fillna(int(0))
    # Calculate the sum of each rows and create a new dataframe with the results
    rsi_tickers_up = pd.DataFrame(rsi_tickers_up.sum(axis=1))
    # Rename the column of the dataframe
    rsi_tickers_up.columns=['sum']
    # Retrieve all tickers with a sum different than 0
    rsi_tickers_up = rsi_tickers_up.loc[(rsi_tickers_up['sum']!= 0),:]
    # Print the shape to see how many tickers we have
    print(f'Number of tickers below the lower_level {rsi_tickers_up.shape[0]}')

    # Retrieve the tickers which had a RSI below the lower_level over the timeperiod
    rsi_tickers_dn = rsi_df_date[condition_dn].fillna(int(0))
    # Calculate the sum of each rows and create a new dataframe with the results
    # This step is to actually retrieve tickers that match the requirement at least once over the timeperiode
    # We don't actually need to calculate the sum - it is used as a flag
    rsi_tickers_dn = pd.DataFrame(rsi_tickers_dn.sum(axis=1))
    # Rename the column of the dataframe
    rsi_tickers_dn.columns=['sum']
    # Retrieve all tickers with a sum different than 0
    rsi_tickers_dn = rsi_tickers_dn.loc[(rsi_tickers_dn['sum']!= 0),:]
    # Print the shape to see how many tickers we have
    print(f'Number of tickers below the lower_level {rsi_tickers_dn.shape[0]}')
    
    # Combine both dataframe to create a list of unique tickers
    combined_rsi_tickers = pd.concat([rsi_tickers_up,rsi_tickers_dn],axis=1, join='outer')
    tickers = list(combined_rsi_tickers.index.values)

    return tickers

# Method test
df = rsi_df
start_date = '2020-01-23'
end_date = '2020-02-11'
lower_level = 30
upper_level = 70 
rsi_tickers = rsi_filter(df, lower_level, upper_level, start_date, end_date)
print(rsi_tickers)

Number of tickers below the lower_level 102
Number of tickers below the lower_level 103
['A', 'AAL', 'ABMD', 'ADBE', 'ADI', 'ADP', 'AEE', 'AEP', 'AJG', 'ALXN', 'AMT', 'AMZN', 'ANET', 'ANTM', 'AON', 'APA', 'APH', 'AWK', 'AXP', 'BAX', 'BDX', 'BEN', 'BF.B', 'BIIB', 'BK', 'BKNG', 'BMY', 'BR', 'BWA', 'BXP', 'CB', 'CCL', 'CF', 'CHRW', 'CL', 'CMA', 'CMS', 'CNC', 'COG', 'COO', 'COP', 'COST', 'CPRT', 'CSCO', 'CTSH', 'CTVA', 'CTXS', 'CVX', 'D', 'DAL', 'DFS', 'DGX', 'DHI', 'DISCK', 'DISH', 'DLR', 'DLTR', 'DUK', 'DVA', 'DVN', 'DXCM', 'EA', 'EBAY', 'ED', 'EMN', 'EOG', 'ETR', 'ETSY', 'EVRG', 'EW', 'EXPD', 'EXR', 'FANG', 'FBHS', 'FE', 'FITB', 'FLT', 'FMC', 'FRT', 'GE', 'GILD', 'GL', 'GWW', 'HAL', 'HBAN', 'HD', 'HES', 'HII', 'HOLX', 'HSY', 'IBM', 'ICE', 'IFF', 'ILMN', 'INTC', 'IP', 'IPGP', 'ISRG', 'IT', 'JBHT', 'JNPR', 'K', 'KEY', 'KEYS', 'KHC', 'KLAC', 'KMB', 'KR', 'KSU', 'LB', 'LDOS', 'LEN', 'LH', 'LMT', 'LNT', 'LW', 'LYB', 'MAA', 'MAR', 'MCHP', 'MCK', 'MCO', 'MDLZ', 'MGM', 'MKC', 'MLM', 'MMM', 'MPC

In [43]:
# MACD filter
# Retrieving all the tickers that match the desired requirements : 
    
def macd_filter_buy(df, start_date, end_date):
    
    '''MACD Filter
        Returns list of tickers for which the MACD crossed the MACD_signal over the timeperiod specified
    
        Parameters
        ----------
        df : pd.DataFrame - dataframe to be processed
        start_date : str() - string with date in following format YYYY-MM-DD
        end_date : str() - string with date in following format YYYY-MM-DD
    
        Returns
        -------
        list_tickers_buy : list[] - list of tickers for which the MACD crossed the MACD_signal upward over the timeperiod
    '''

    # Filter the rsi dataframe generated by the "rsi_table" method
    macd_df_date = df.loc[start_date : end_date, :]
    
    # Split the Dataframe into two distinct ones based on the indicator
    signal = macd_df_date['MACD_Signal']
    macd   = macd_df_date['MACD']
    
    # Declare the list to store the tickers
    list_tickers_buy = []

    # Iterate through each cells of the dataframe to test the condition and append the list if condition matched
    for i in range(len(signal)):
        if i != 0:
            for x in range(len(signal.columns)):
                if macd.iloc[i, x] > signal.iloc[i, x] and macd.iloc[i-1, x] <= signal.iloc[i-1, x]:
                    if macd.columns[x] not in list_tickers_buy:
                        list_tickers_buy.append(macd.columns[x])
   
    return list_tickers_buy

def macd_filter_sell(df, start_date, end_date):
    
    '''MACD Filter
        Returns list of tickers for which the MACD crossed the MACD_signal over the timeperiod specified
    
        Parameters
        ----------
        df : pd.DataFrame - dataframe to be processed
        start_date : str() - string with date in following format YYYY-MM-DD
        end_date : str() - string with date in following format YYYY-MM-DD
    
        Returns
        -------
        list_tickers_sell : list[] - list of tickers for which the MACD crossed the MACD_signal downward over the timeperiod
    '''
     # Filter the rsi dataframe generated by the "rsi_table" method
    macd_df_date = df.loc[start_date : end_date, :]
    
    # Split the Dataframe into two distinct ones based on the indicator
    signal = macd_df_date['MACD_Signal']
    macd   = macd_df_date['MACD']
    
    # Declare the list to store the tickers
    list_tickers_sell = []


    # Iterate through each cells of the dataframe to test the condition and append the list if condition matched
    for i in range(len(signal)):
        if i != 0:
            for x in range(len(signal.columns)):
                if macd.iloc[i, x] < signal.iloc[i, x] and macd.iloc[i-1, x] >= signal.iloc[i-1, x]:
                    if macd.columns[x] not in list_tickers_sell:
                        list_tickers_sell.append(macd.columns[x])
    
    return list_tickers_sell
    
# Method test
df = combined_macd
start_date = '2020-01-23'
end_date = '2020-02-11'
macd_tickers_buy = macd_filter_buy(df, start_date, end_date)
macd_tickers_sell = macd_filter_sell(df, start_date, end_date)

print(len(macd_tickers_buy))
print(len(macd_tickers_sell))


295
305


### Fundamental

In [12]:
# Please start with PE, EPS, and Dividned Info.
# your method must take entire dataframe and add a PE, EPS, and Dividned Info columns to it
# please keep in mind that the dataframe will contain multiple tickers
# follow specs for method writing outlined below, feel free to expand and improve
# document your method with docstring
# document theory for your method in readme (see the README.md for example)

# Method test

### Traditional

In [13]:
# Please start with calculating the Sharpe Ratio Calculation
# your method must take entire dataframe and return a dataframe with ticker and sharpe ratio for the given time
# please keep in mind that the dataframe will contain multiple tickers
# follow specs for method writing outlined below, feel free to expand and improve
# document your method with docstring
# document theory for your method in readme (see the README.md for example)

# Method test

## Visualization

In [14]:
# your code here

## Dashboard

In [15]:
# your code here